In [1]:
import pandas as pd
import numpy

In [2]:
def tokenize(x):
    return x.strip().split(' ')

In [3]:
def load_file(file_name):
    with open(data_file) as d:
        for line in d:
            yield tokenize(line)
        

In [4]:
data_file = 'data/wikitext-2/train.txt'
f = load_file(data_file)

In [5]:
def get_tokens(file):
    tokens = {}
    for sent in file:
        for token in sent:
            try:
                tokens[token] += 1
            except:
                tokens[token] = 1
    return tokens

In [6]:
%%time 
tokens = get_tokens(f)
print(len(tokens))

33278
CPU times: user 356 ms, sys: 3.6 ms, total: 360 ms
Wall time: 450 ms


In [7]:
import math
def norm_probs(tokens):
    s = sum(tokens.values())
    return {k : round(math.log(v/float(s)),4) for k, v in tokens.items()}
def sort_probs(probs):
    return sorted(probs.items(), key=lambda x:x[1], reverse=True)

In [8]:
probs = norm_probs(tokens)
sorted_probs = sort_probs(probs)

In [9]:
sorted_probs[100:130]

[('000', -7.2633),
 ('song', -7.2654),
 ('before', -7.2675),
 ('being', -7.2793),
 ('years', -7.2892),
 ('5', -7.3246),
 ('series', -7.3491),
 ('album', -7.3491),
 ('second', -7.3705),
 ('through', -7.372),
 ('can', -7.3759),
 ('both', -7.3774),
 ('4', -7.3867),
 ('part', -7.3883),
 ('then', -7.3922),
 ('including', -7.3938),
 ('New', -7.397),
 ('/', -7.4009),
 ('against', -7.4017),
 ('number', -7.4025),
 ('became', -7.4057),
 ('film', -7.4073),
 ('known', -7.4105),
 ('well', -7.4153),
 ('—', -7.4177),
 ('many', -7.4439),
 ('there', -7.4463),
 ('no', -7.4513),
 ('United', -7.4555),
 ('century', -7.4853)]

In [10]:
a = "Hello"
a[3:10]

'lo'

In [15]:

# def is_stack_completed(stack, token):
#     return all([len(''.join(x)) == len(token) for x in stack]) and len(stack) > 0


def is_stack_completed(stack, token):
    new_stack = []
    for s in stack:
        new_s = [x if i == 0 else x[1:] for i, x in enumerate(s)]
        new_stack.append(new_s)
    #print(stack, '/', new_stack)
    return all([len(''.join(x)) == len(token) for x in new_stack]) and len(new_stack) > 0

def get_grams(token, position, start_from=2, max_gram = 4):
    return list(set([token[position:position+x] for x in range(start_from, max_gram + 1)]))


def print_stack(stack, level = 0):
    cleaved = {}
    
    for x in stack:
        if x[0] not in cleaved:
            cleaved[x[0]] = []
        if len(x) > 1:
            cleaved[x[0]].append(x[1:])
    
    for k, v in cleaved.items():
        print(level*"-", k)
        if len(v) > 0:
            print_stack(v, level+1)
    
 
def process_token(token, max_gram = 4):
    stack = []
    counter = 0
    pairs = {}
    while is_stack_completed(stack, token) == False:
        #print(counter, stack)
        if len(stack) == 0:
            stack += [[x] for x in get_grams(token, 0)]
        p_stack = []
        for i, element in enumerate(stack):
            position = len(''.join( [x if i == 0 else x[1:] for i, x in enumerate(element)])) - 1
            grams = get_grams(token, position)
            #print(i, '/', element, '/', stack, '/', p_stack, grams)
            for g in grams:
                if len(g) > 0:
                    p = element[-1]
                    if len(p)> 1 and len(g) > 1:
                        p_stack.append(element + [g])
                        pair = (p, g)
                        if counter > 15:
                            print(token, "Something wrong here", stack,  p_stack, grams)
                            return None
                        #yield pair
                        if pair not in pairs:
                            yield pair
                            pairs[pair] = 0
    #                     if pair not in pairs:
#                         pairs[pair] = 0
#                     pairs[pair] += 1
        stack = p_stack
        #print_stack(stack)
        counter += 1
       
    #return pairs
        
    

In [16]:
list(process_token('apple'))

[('ap', 'pple'),
 ('ap', 'ppl'),
 ('ap', 'pp'),
 ('appl', 'le'),
 ('app', 'pl'),
 ('app', 'ple'),
 ('ppl', 'le'),
 ('pp', 'pl'),
 ('pp', 'ple'),
 ('pl', 'le')]

In [17]:
words = [x[0] for x in sorted_probs if len(x[0]) > 6]

In [18]:
sample_words = words[:15000]

In [19]:
%time
# token = 'insurmountable'
# #pairs = process_token(token)


pairs = {}
unigrams = {}

for t, token in enumerate(sample_words):
    for pair in process_token(token):
        if pair != None:
            if pair not in pairs:
                pairs[pair] = 0
            pairs[pair] += 1

            for k in pair:
                kp = (k, )
                if kp not in pairs:
                    pairs[kp] = 0
                pairs[kp] += 1
    if t % 1000 == 0:
        print(t, "Done !!")



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs
0 Done !!
1000 Done !!
2000 Done !!
3000 Done !!
4000 Done !!
5000 Done !!
6000 Done !!
7000 Done !!
8000 Done !!
9000 Done !!
10000 Done !!
11000 Done !!
12000 Done !!
13000 Done !!
14000 Done !!


In [20]:
pairs, sum(pairs.values())

({('be', 'etw'): 1,
  ('be',): 981,
  ('etw',): 34,
  ('be', 'etwe'): 1,
  ('etwe',): 6,
  ('be', 'et'): 14,
  ('et',): 2455,
  ('bet', 'twe'): 1,
  ('bet',): 50,
  ('twe',): 30,
  ('bet', 'twee'): 1,
  ('twee',): 9,
  ('bet', 'tw'): 1,
  ('tw',): 205,
  ('betw', 'ween'): 1,
  ('betw',): 3,
  ('ween',): 9,
  ('betw', 'wee'): 1,
  ('wee',): 37,
  ('betw', 'we'): 1,
  ('we',): 654,
  ('etw', 'ween'): 2,
  ('etw', 'wee'): 2,
  ('etw', 'we'): 2,
  ('etwe', 'een'): 2,
  ('een',): 222,
  ('etwe', 'ee'): 2,
  ('ee',): 1198,
  ('et', 'twe'): 2,
  ('et', 'twee'): 2,
  ('et', 'tw'): 8,
  ('twe', 'een'): 2,
  ('twe', 'ee'): 3,
  ('twee', 'en'): 2,
  ('en',): 8258,
  ('tw', 'ween'): 2,
  ('tw', 'wee'): 3,
  ('tw', 'we'): 8,
  ('wee', 'en'): 3,
  ('we', 'een'): 3,
  ('we', 'ee'): 10,
  ('ee', 'en'): 55,
  ('thr', 'roug'): 4,
  ('thr',): 135,
  ('roug',): 84,
  ('thr', 'rou'): 4,
  ('rou',): 370,
  ('thr', 'ro'): 16,
  ('ro',): 4051,
  ('th', 'hr'): 33,
  ('th',): 2374,
  ('hr',): 437,
  ('th', 'hro

In [21]:
sorted_pairs = sorted(pairs.items(), key = lambda x : x[1], reverse=True)

In [22]:
sorted_pairs[:10]

[(('in',), 12575),
 (('ti',), 11861),
 (('er',), 11736),
 (('on',), 9256),
 (('at',), 9246),
 (('te',), 8353),
 (('en',), 8258),
 (('re',), 8017),
 (('es',), 7410),
 (('an',), 7317)]

In [27]:
import math
def calc_assocs(pairs):
    
    total = sum(pairs.values())
    assocs = {}
    for p in pairs:
        if len(p) > 1:
            x, y = p
            cXY = pairs[(x, y)]
            cX = pairs[(x,)]
            cY = pairs[(y,)]

            p_x_given_y = float(cXY) / cY 
            p_x_and_not_y = float(cX - cXY)
            p_not_y = total - cY
            p_x_given_not_y = p_x_and_not_y / float(p_not_y)       


            p_y_given_x = float(cXY) / cX 
            p_y_and_not_x = float(cY - cXY)
            p_not_x = total - cX
            p_y_given_not_x = p_y_and_not_x / float(p_not_x)

            delta_p = p_x_given_y - p_x_given_not_y
            delta_p_r = p_y_given_x - p_y_given_not_x
            #delta_p = delta_p / 2.0
#             delta_p, delta_p_r = 0, 0
#             try:
#                 delta_p = -math.log(p_x_given_y) + math.log(p_x_given_not_y)
#                 delta_p_r = -math.log(p_y_given_x) + math.log(p_y_given_not_x)
#             except:
#                 pass
            #p_x_y = float(cXY) / total
            #p_x, p_y = float(cX)/total, float(cY)/total
            #pmi_x_y = math.log(p_x_y/(p_x*p_y))
            #delta_p = pmi_x_y + math.log(p_x_y)
            #delta_p_r = delta_p
            assocs[(x,y)] = (delta_p, delta_p_r)
    return assocs
        

In [28]:
assocs  = calc_assocs(pairs)

In [29]:
sorted_assocs = sorted(assocs.items(), key = lambda x : sum(x[1]), reverse=True)

In [30]:
sorted_assocs[-30:]

[(('er', 'rib'), (-0.0024655732717530233, -5.980458846381629e-05)),
 (('min', 'nt'), (-0.00031194909723811606, -0.0022163872744239976)),
 (('li', 'ir'), (-0.002042719412083729, -0.0005134410734723548)),
 (('ati', 'it'), (-0.001155873839178435, -0.0014063980364033644)),
 (('ene', 'en'), (-0.00016687258347305954, -0.0024423689982903123)),
 (('er', 'rep'), (-0.0025508877824370075, -6.340418942233859e-05)),
 (('es', 'sio'), (-0.0024569619773144747, -0.0002498627221377789)),
 (('ci', 'ic'), (-0.0008218040178037772, -0.0019436332239488427)),
 (('fo', 'on'), (-0.00026206110650561765, -0.0025887603500479696)),
 (('er', 'ret'), (-0.0027941029117358965, -7.471722100626587e-05)),
 (('er', 'rec'), (-0.002737000369070692, -0.0001437779262205681)),
 (('ti', 'isc'), (-0.0028163686577827455, -7.356843838690121e-05)),
 (('ple', 'er'), (-7.831682196478817e-05, -0.002864456347415457)),
 (('ai', 'io'), (-0.000643558512451045, -0.0024476854514922652)),
 (('cha', 'at'), (-0.00017687709201703026, -0.00292830

In [31]:
new_sample = words[13000:13500]

In [91]:
def is_stack_completed2(stack, token):
    r = True
    if len(stack) == 0:
        return False
    for element in stack:
        l = len(''.join([x[0] if i==0 else x[0][1:] for i,x in enumerate(element)]))
        if l != len(token):
            return False
    return r
    #return all([len(''.join(x)) == len(token) for x in stack]) and len(stack) > 0


def split_token(token, max_gram = 4, thresh=0.001):
    stack = []
    counter = 0
    pairs = {}
    while is_stack_completed2(stack, token) == False:
        #print(counter, stack)
        if len(stack) == 0:
            stack += [[(x, x)] for x in get_grams(token, 0)]
        p_stack = []
        #print(stack)
        for i, element in enumerate(stack):
            position = len(''.join([x[0] if i==0 else x[0][1:] for i,x in enumerate(element)])) - 1
            if position == len(token) - 1:
                #print('~~@@', element, position)
                p_stack.append(element)
            grams = get_grams(token, position)
            for g in grams:
                if len(g) > 1:
                    #print(element, g, '//', p_stack)
                    #p_stack.append(element + [g])
                    pair = (element[-1][0], g)
                    if pair in assocs  and assocs[pair][0] > thresh and assocs[pair][1] > thresh:
                        #print(pair, assocs[pair])
                        p_stack.append(element + [(g, element[-1][1] + g[1:])])
                    else:
                        p_stack.append(element + [(g, g[1:])])
#                 elif len(g) > 0:
#                     p_stack.append(element + [(g, element[-1][1] + g)])
                        #yield pair
        stack = p_stack
        #print('~~', stack)
        #print(p_stack)
        #print_stack(stack)
        
        counter += 1
        #break
    return stack
    #return pairs
        
def get_scores(stack):
    scores = []
    joins = []
    for split in stack:
        score = 0 
        join = []
        
        for a, b in zip(split, split[1:]):            
            if b[1].startswith(a[1]) == False:
                join.append(a[1])
            if (a[0], b[0]) in assocs:
                score += sum(assocs[(a[0], b[0])]) / 2 
                
        if len(join) == 0 or (split[-1][1].startswith(join[-1]) == False):
            join.append(split[-1][1])
        scores.append((join, score))
    return scores

In [92]:
split_token('apple')

[[('ap', 'ap'), ('pple', 'ple')],
 [('ap', 'ap'), ('ppl', 'appl'), ('le', 'apple')],
 [('ap', 'ap'), ('pp', 'app'), ('pl', 'appl'), ('le', 'apple')],
 [('ap', 'ap'), ('pp', 'app'), ('ple', 'apple')],
 [('appl', 'appl'), ('le', 'e')],
 [('app', 'app'), ('pl', 'appl'), ('le', 'apple')],
 [('app', 'app'), ('ple', 'le')]]

In [93]:
def get_max_split(token):
    stack = split_token(token)
    scores = get_scores(stack)
    #print(scores)
    return max(scores, key=lambda x: x[1])

In [94]:
split = get_max_split(words[3000])
split

(['Classic'], 0.2320549523489015)

In [95]:
for w in new_sample:
    print(w, get_max_split(w))

hugging (['huggi', 'ng'], 0.4745634842336762)
defaced (['de', 'faced'], 0.14035851350327516)
armlets (['arm', 'lets'], 0.2703923385125039)
engraved (['engr', 'a', 'ved'], 0.33019002671701053)
dilapidated (['di', 'lapida', 'ted'], 0.53506141035287)
pedestal (['pede', 'stal'], 0.18801870885913255)
enshrined (['en', 'shr', 'ined'], 0.33787718007461226)
preparedness (['prepar', 'edn', 'ess'], 0.46518499636353194)
parapet (['par', 'ape', 't'], 0.15835777573108437)
Thought (['Thought'], 0.3539229761976216)
Noisescapes (['Noisesc', 'apes'], 0.4479072183380498)
Revolver (['Revolver'], 0.34314409359894565)
Physicist (['Physicist'], 0.331046017019074)
impenetrable (['impen', 'etr', 'able'], 0.4587171666122843)
clarifying (['cla', 'rifying'], 0.548513357779081)
leftover (['le', 'ftove', 'r'], 0.49591501200704124)
nightmare (['nightmar', 'e'], 0.527335364281794)
hardship (['hardship'], 0.326113585080396)
recharge (['recharge'], 0.2840771394415267)
chaotic (['cha', 'oti', 'c'], 0.38876964016189364)